# Class 2 - Types, Functions, and Modules

Today, we'll see how Julia lets us rapidly develop code like Python or Matlab, and also how we can drill down and be specific as in C/Fortran.  Sometimes these two styles are not mutually exclusive.

First, we'll demonstrate how to quickly write a function.  For simplicity, we'll just consider the example

$$f(x,y) = x + y^2$$

In [ ]:
function demofunc(x,y)
    x + y^2
end

In [ ]:
# a more concise way to write short functions
demofunc(x,y) = x + y^2

In [ ]:
# we can now call our demo function on a variety of inputs
@show demofunc(2,5)
@show demofunc(2, 5.0)
@show demofunc(2, randn(2,2))
@show demofunc('h', 2)
;

As we see, you can write a function without worrying too much about details.  However, worrying about the details helps us write code that is faster, and more importantly, correct.  Today, we'll discuss some of the details that are good to understand, even if you don't use them explicitly in your code.

## Types

Julia's documentation for types can be found [here](https://docs.julialang.org/en/stable/manual/types/).

If you have primarily programmed in Python, MATLAB, or R, you likely haven't had to think too much about types before.  For example, just about everything in MATLAB is assumed to be a ```double``` in C, or ```Float64``` in Julia.  This makes it easier to write code, but ultimately can sometimes make it difficult to write code that is fast and memory efficient.

If you have experience in C, C++, or Fortran, you probably already know something about types - today we'll see how Types work in Julia.

### Type Basics

When you use a type in a programming language, you are telling the computer
* How a variable should be stored
* Something about how you intend to use it

Even if you haven't seen types before in programming, you should already have some intution from mathematics - consider for example, integers $Z$, real numbers $R$, and complex numbers $C$ - you can think of these as types
* $Z$, $R$, and $C$ are closed under addition and subtraction (+ takes two inputs and has a deterministic output)
* You can divide in $R$ and $C$ (except for 0), but not in $Z$ (3 / 2?)
* You know something about representing an element on a piece of paper: $Z$ (no decimal, but maybe a symbol $-$), $R$ (perhaps you need a decimal), and $C$, (you can use two real numbers and a symbol $i$)
* You can represent an integer as a real number, and a real number as a complex number, but not the other ways around (this is an example of type promotion)
* Certain functions have outputs in one of these sets.  For example, a square root is generally complex (an example of output type)

Using types in programming allows you to share your knowledge about all these issues with a computer.

We didn't use types in ```demofunc``` which made programming very easy for us.  However, this means that the compiler/interpreter can't assume too much about our inputs $x$ and $y$, or what we're going to do with them. 

Types can be useful for several reasons
* They give the complier/interpreter information that can be used to make code faster and more memory efficient
* Figuring things out is done at complie time, not run time
* They help you (the programmer) keep track of what code should be doing
* They prevent the code from being used in unintended ways ('h' + 2^2?)


### Types in Julia


Types in Julia come in several flavors.  For example, the following types may look somewhat familiar

In [ ]:
@show typeof(4)
@show typeof(0x4)
@show typeof(3.2)
;

Everything has a type in Julia, even types

In [ ]:
@show typeof(Float64)
@show typeof(DataType)
@show supertype(DataType)
@show typeof(+)
@show supertype(typeof(+))
@show typeof(randn(3,3))
;

Types like Float64 and Int64 actually hold data (they are [primitive types](https://docs.julialang.org/en/stable/manual/types/#Primitive-Types-1) in Julia).  Every type in Julia has a "super" type. The type structure is a tree - Any is the root.

In [ ]:
@show supertype(Float64)
@show supertype(AbstractFloat)
@show supertype(Real)
@show supertype(Number)
@show supertype(Any)

@show isbits(Float64) # note primitive types used to be called bits types - think of this as isprimitive
;

you can also go down the tree by seeing subtypes of a supertype

In [ ]:
subtypes(AbstractFloat)

In [ ]:
# Types that actually hold data don't have subtypes
subtypes(Float64)

Floating point numbers ("Floats") are a way to represent real numbers up to a certain accuracy on a computer.  The 16, 32, and 64 on ```Float16```, ```Float32```, ```Float64``` refer to the number of bits (1s and 0s) used to store the type on the computer.

In [ ]:
# Get machine precision for a floating point type
@show eps(Float16)
@show eps(Float32)
@show eps(Float64)
;

Many modern machines have 64-bit architectures, so in Julia the default is to use a 64-bit size to store numbers.  This means that floats are represented by ```Float64```, that integers by ```Int64```, and unsigned integers by ```UInt64```.  For more information on working with integers and floats see Julia's [documentation](https://docs.julialang.org/en/release-0.4/manual/integers-and-floating-point-numbers/).

You can query whether one type is a descendant of another using "<:"

In [ ]:
@show Float64 <: AbstractFloat
@show Float64 <: Any
@show Float64 <: Integer

@show typeof(3.4) == Float64
;

Abstract types are nodes on the tree, but you never instantiate them (we'll talk about why you might want abstract types in functions)

In [ ]:
abstract type cme257abstract end
@show supertype(cme257abstract)
;

Composite types in Julia are declared using the ```struct``` keyword.  (similar to C or MATLAB).  When you create a type, you can specify who its parent is.

In [ ]:
# an abstract type for our real number types
abstract type cme257real <: cme257abstract end

struct cme257int <: cme257real # <: denotes "child of"
    x::Int64 # :: tells us exactly what type x should be
end
struct cme257float <: cme257real
    x::Float64
end
@show supertype(cme257int)
@show supertype(cme257float)
;

When you create a composite type, a constructor function is automatically created based on the values of the type. To instantiate a member of the type:

In [ ]:
y = cme257int(4)
@show y
@show y.x
;

Structs are not by default mutable, meaning, you can't change the values of fields once they are set.  To make a mutable struct, use the ```mutable``` keyword.  Mutable types are obviously more flexible, but often suffer a performance penalty.

Note, that in the past, ```type``` could be used instead of ```mutable struct```, and ```immutable``` could be used instead of ```struct```.  This old syntax still works, but is being phased out, so we're learning the current syntax.  You may still see the old syntax used in existing code.

In [ ]:
mutable struct cme257mutable
    a::Int64
    b::Float64
end

struct cme257immutable
    a::Int64
    b::Float64
end

x = cme257mutable(1, 1.0)
@show x
x.a = 2
@show x

y = cme257immutable(1, 1.0)
@show y
y.a = 2
@show y
;

Types can be parameterized (similar to C++ templates).  Think of the parameter as a variable that can be filled in later.  The parameterized type can be inferred by the type of the arguments in the constructor, or made explicit.

In [ ]:
struct cme257par{T} <: cme257abstract
    val::T 
end
;

In [ ]:
@show y = cme257par(3.5) # type can be inferred
@show z = cme257par{Float64}(3) # explicit type
;

You can modify the behavior of the default constructor in the type definition.  You can also add additional constructors for different types of input.  See more in the [documentation](https://docs.julialang.org/en/stable/manual/constructors/#man-constructors-1)

In [ ]:
# we'll use this for rings of integers mod N
struct cme257ff{N, T<:Integer} #<: cme257abstract
    val::T
    # override the default constructor to store things mod N
    cme257ff{N,T}(val::T) where {N,T<:Integer} = new(mod(val,N))
end

# You'll also see the N convention in Julia with types like Array{T,N}, where T is a type and N is a number

# This will create constructors where the type of the value is inferred
cme257ff{N}(val::T) where {N, T} = cme257ff{N,T}(val)
# this will create constructors where signed integers are converted to unsigned integers
cme257ff{N,T}(val::T) where {N, T<:Signed} = cme257ff{N}(Unsigned(val))
# Note that the above are examples of creating functions

@show x = cme257ff{7, UInt8}(UInt8(12)) 
@show x2 = cme257ff{7}(UInt8(5))
@show x3 = cme257ff{5}(UInt(3))
@show x4 = cme257ff{11}(13)
;

Types constructed with parameters are called UnionAll types.  They are not DataTypes, since they can not hold data until their parameters are specified.

One nice feature of using parametric types is that it allows you to ignore details that are largely irrelevant.  In the above example, we don't need to worry about what how the ```val``` parameter is actually stored (that is, what the type is), as long as it is an integer.  If someone who uses the type cares sufficiently, they can force the issue with an explicit constructor - otherwise, the issue can mostly be ignored (as with ```x4```).  We'll see more about the power of parametric types when we talk about functions.

For the purposes of this class, you don't need to know much about UnionAll types beyond the basics (the above example is more advanced).  For more information about UnionAll types, and what you can do with the ```where``` keyword, check out the [documentation](https://docs.julialang.org/en/stable/manual/types/#UnionAll-Types-1).

In [ ]:
@show typeof(cme257ff)
@show typeof(cme257ff{5,UInt8})
@show supertype(cme257ff{5,UInt8})
;

Some of Julia's basic types such as ```Int64``` are known as primitive types (as opposed to composite types via ```struct```).  An example of creating a primitive type - you'll have to define quite a few functions by yourself, since you don't even get a constructor.  This is the only time we'll create a primitive type in this course.

In [ ]:
primitive type cme257prim <: cme257abstract 8 end

In [ ]:
a = Core.Intrinsics.bitcast(cme257prim, UInt8(5)) # bit cast to put bits in your type

## Break for Worksheet 1

## Functions

Functions map a tuple of arguments to an output. For example, the following function maps two inputs to their sum.

In [ ]:
function cme257_sum(x, y)
    return x + y
end
@show z = cme257_sum(1,2)
;

Julia uses multiple dispatch in functions.  This means you can define two functions with the same name that will behave differently based on the type of the input.  A definition of one possible behavior for a function is [called a method](https://docs.julialang.org/en/stable/manual/methods/).

In [ ]:
# this may not work if you have a 32-but architecture
function yell_my_type(x::Int64)
    println("I'M AN INT64!")
end
function yell_my_type(x::Float64)
    println("I'M A FLOAT64!")
end
yell_my_type(3)
yell_my_type(3.4)
;

The ```x::Int64``` syntax is an example of a type annotation - it tells the compiler exactly what the type of the input should be.  This is analgous to C's ```int x``` declaration.

If you've only defined your function for particular types, you may see an error like this:

In [ ]:
yell_my_type(3//4)

Errors can be bad if you expect the function to behave a certain way, or good if they indicate that someone is using your function incorrectly.  If you want the same behavior for all types that share an ancestor, you can use "<:"

In [ ]:
function yell_my_type{T <: Number}(x::T)
    println("I'M A NUMBER!")
end
yell_my_type(3//4)
yell_my_type(3.4) # note that the more specific function declaration is used
;

If you want to try to cover all possible inputs, you can just leave off type annotations.  Your more specific methods will still be used for the relevant inputs.

In [ ]:
function yell_my_type(x)
    println("I DON'T KNOW MY TYPE!")
end
yell_my_type(yell_my_type) # everything in Julia has a type.  Even functions.
yell_my_type(3//4)
yell_my_type(3.4)
;

You can also add new methods to Julia's built in functions.  If you do this, make sure to import the function first (default functions are in the Base module), or Julia will think that your definition is the only one out there.

In [ ]:
import Base.+
function +{T<:cme257real}(a::T, b::T)
    return T((a.x+b.x))
end
a = cme257int(3)
b = cme257int(4)
@show a + b
;

You can also annotate the return type of a function

In [ ]:
function cme257_sum(x::Int64, y::Int64)::Float64
   return x+y 
end

cme257_sum(1, 2)

Once you've created your own types, you may also wish have custom printing for things like the ```@show``` macro.  See the [documentation](https://docs.julialang.org/en/stable/manual/types/#Custom-pretty-printing-1) for reference.

In [ ]:
# pretty printing for our cme257ff type with underlying storage mentioned
Base.show{N,T}(io::IO, x::cme257ff{N,T}) = print(io, x.val, " mod ", N, " (", T, ")")

# 7 mod 5
x = cme257ff{5}(7)
@show x
;

## Break for Worksheet 2

## More on Types and Functions

Sometimes you may wish to have arguments to a function that default to certain values.

In [ ]:
function say(str="hello")
    println("$str")
end
say()
say("goodbye")
;

You can also add key-word arguments (to the right of a semicolon ```;```).  These must come with default assignments

In [ ]:
function cme257kw(a::Float64; b::Int64=10, c::Int64=20)
    println("a = $a, b = $b, c=$c")
end

cme257kw(1.0)
cme257kw(2.0, c=11)
cme257kw(3.0, c=10, b=5)
;

You can also add methods to constructor functions for types.  This is useful if you want default constructors, or want to be able to construct a type in a variety of ways.

In [ ]:
cme257int() = cme257int(0)
@show a = cme257int()
;

### Checking Input
Sometimes, you will want to check that certain conditions on an input are met before executing a function.  The ```@assert``` macro is useful for this.

In [ ]:
function cme257dot{T}(x::Vector{T}, y::Vector{T})
    @assert length(x) == length(y)
    return x.*y
end

x = randn(5)
y = randn(5)
@show cme257dot(x,y)

y = randn(7) # now x and y have different lengths
@show cme257dot(x,y)
;

### Type Conversion

What if you have defined a special integer type (such as cme257int), and want to add a method to "/" to implement division?  The true answer may be something that is not an integer, so you may wish to convert to a cme257float.

In [ ]:
import Base.convert
function convert(::Type{cme257float}, a::cme257int) 
    return cme257float(a.x)
end

@show a = cme257int(5)
@show convert(cme257float, a)
;

now since we can convert cme257ints to cme257floats, we can define division

In [ ]:
import Base./
function /(a::cme257float, b::cme257float)
    return cme257float(a.x / b.x) 
end

function /(a::cme257int, b::cme257int)
    return convert(cme257float, a) / convert(cme257float, b) 
end

a = cme257int(5)
b = cme257int(4)
@show a/b
;

Note that Julia's built in arithmetic types and operators already have conversion defined and implemented where necessary.  You mostly need to know about this if you're defining your own types.

### Promotion

Suppose you want to add a cme257int to a cme257float.  What would the correct type be?  Since integers are a subset of the reals, the natural answer is a cme257float.  In order to achieve this, we should promote a cme257int to a cme257float, on which addition is defined.

In [ ]:
import Base.promote_rule
function convert(::Type{cme257float}, a::cme257float) 
    return a
end

# the following tells us how to promote types when there is a cme257int and cme257float involved
promote_rule(::Type{cme257float}, ::Type{cme257int}) = cme257float
promote_rule(::Type{cme257int}, ::Type{cme257float}) = cme257float

function +{T1<:cme257real, T2<:cme257real}(a::T1, b::T2)
    T = promote_rule(T1, T2)
    return convert(T,a) + convert(T,b)
end

a = cme257int(1)
b = cme257float(1.0)

@show a+b
@show b+a
;

We can see how to make our cme257ff operations more robust to underlying storage type

In [ ]:
import Base.*, Base.+
function *{N, T1, T2}(a::cme257ff{N,T1}, b::cme257ff{N,T2})
    T = promote_type(T1, T2)
    cme257ff{N,T}(T(mod(a.val * b.val, N)))
end

function +{N,T1, T2}(a::cme257ff{N,T1}, b::cme257ff{N,T2})
    T = promote_type(T1, T2)
    cme257ff{N,T}(T(mod(a.val + b.val,N)))
end

x = cme257ff{5}(UInt8(2))
@show x
y = cme257ff{5}(UInt16(4))
@show y
@show x*y # 2 * 4 = 8 = 3 mod 5
@show x+y # 2 + 4 = 6 = 1 mod 5
;

## Modules

Modules are useful for packaging (modularizing) functionality that you create.  This allows you to re-use code, or automatically import functions that you like to use when you start a new Julia session.  We'll see later how this is used in packages, but for now we'll just include a script.

In [ ]:
include("cme257mod.jl") # you need to provide an accurate absolute or relative path
using cme257mod # exports the functions listed in cme257mod

In [ ]:
speak()
@show x = ModType(5)
not_exported()
;

In [ ]:
cme257mod.not_exported()

## Break for Worksheet 3

## Further Reading

If you think types, templates, and functions are interesting, I recommend the books
* "From Mathematics to Generic Programming" by Stepanov and Rose (online version at [Stanford Library](https://searchworks.stanford.edu/view/10724750))
* "Elements of Programming" by Stepanov and McJones (online version at [Stanford Library](https://searchworks.stanford.edu/view/8262498))
